In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import from_json
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import time as timer
import argparse
import datetime
import json
from pyspark.sql.functions import col, abs, mean, expr, substring, udf
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

import random
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
#import lightgbm as lgb

os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.driver.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell'

schema = StructType(
    [
        StructField("num_date_time", StringType()),
        StructField("건물번호", StringType()),
        StructField("일시", StringType()),
        StructField("기온(C)", StringType()),
        StructField("강수량(mm)", StringType()),
        StructField("풍속(m/s)", StringType()),
        StructField("습도(%)", StringType()),
        StructField("일조(hr)", StringType()),
        StructField("일사(MJ/m2)", StringType()),
        StructField("전력소비량(kWh)", StringType()),
    ]
)

schema2 = StructType(
    [
        StructField("num_date_time", StringType()),
        StructField("건물번호", StringType()),
        StructField("일시", StringType()),
        StructField("기온(C)", StringType()),
        StructField("강수량(mm)", StringType()),
        StructField("풍속(m/s)", StringType()),
        StructField("습도(%)", StringType()),
    ]
)

schema3 = StructType(
    [
        StructField("건물번호", StringType()),
        StructField("건물유형", StringType()),
        StructField("연면적(m2)", StringType()),
        StructField("냉방면적(m2)", StringType()),
        StructField("태양광용량(kW)", StringType()),
        StructField("ESS저장용량(kWh)", StringType()),
        StructField("PCS용량(kW)", StringType()),
    ]
)



print("FILES IN THIS DIRECTORY")
print(os.listdir(os.getcwd()))



def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

#config.json 파일 읽기
with open("config.json", "r") as f:
   config = json.load(f)

jar_urls = ",".join(config["KAFKA_JAR_URLS"])
repartition_num = config["NUM_EXECUTORS"] * config["EXECUTOR_CORES"] * 2
# SparkSession 생성
spark = (
    SparkSession.builder.master("spark://spark-master-service:7077")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.host", "10.42.2.18")
    .config("spark.driver.port", "39337")
    .config("spark.cores.max", "32")
    .config("spark.network.timeout", "600s")
    .config("spark.executor.instances", config["NUM_EXECUTORS"])
    .config("spark.executor.cores", config["EXECUTOR_CORES"])
    .config("spark.executor.memory", config["EXECUTOR_MEMORY"])   
    .config("spark.driver.memory", "30g")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size","20g")
    
    .config("spark.defaul.parallelism", repartition_num)
    .config("spark.sql.shuffle.partitions", repartition_num)
    .config("spark.driver.extraJavaOptions", "--illegal-access=permit")
    .config("spark.executor.extraJavaOptions", "--illegal-access=permit")
    .config("spark.jars", jar_urls)  # JAR 파일 포함
    .appName("asdf")
    .getOrCreate()
)

sc = spark.sparkContext
sc.setLogLevel("ERROR")

print("Current Spark configuration:")
for key, value in sorted(sc._conf.getAll(), key=lambda x: x[0]):
    print(f"{key} = {value}")

FILES IN THIS DIRECTORY
['.bashrc', '.bash_logout', '.profile', '.ipython', '.cache', '.npm', '.bash_history', '.local', '.ipynb_checkpoints', 'config.json', '.jupyter', 'jars', '.conda', '.config', '.wget-hsts', 'work']


24/05/29 09:44:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Current Spark configuration:
spark.app.id = app-20240529094429-0447
spark.app.initial.jar.urls = spark://10.42.2.18:39337/jars/commons-pool2-2.6.2.jar,spark://10.42.2.18:39337/jars/hadoop-client-api-3.3.1.jar,spark://10.42.2.18:39337/jars/kafka-clients-2.8.1.jar,spark://10.42.2.18:39337/jars/spark-sql-kafka-0-10_2.12-3.2.4.jar,spark://10.42.2.18:39337/jars/hadoop-client-runtime-3.3.1.jar,spark://10.42.2.18:39337/jars/htrace-core4-4.1.0-incubating.jar,spark://10.42.2.18:39337/jars/commons-logging-1.1.3.jar,spark://10.42.2.18:39337/jars/spark-streaming-kafka-0-10_2.12-3.2.4.jar,spark://10.42.2.18:39337/jars/jsr305-3.0.0.jar,spark://10.42.2.18:39337/jars/spark-token-provider-kafka-0-10_2.12-3.2.4.jar
spark.app.name = asdf
spark.app.startTime = 1716975868693
spark.cores.max = 32
spark.defaul.parallelism = 96
spark.driver.bindAddress = 0.0.0.0
spark.driver.extraJavaOptions = --illegal-access=permit
spark.driver.host = 10.42.2.18
spark.driver.memory = 30g
spark.driver.port = 39337
spark.exec

In [2]:
# 그냥 가져오기
building_sdf = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", "my-cluster-kafka-bootstrap.kafka.svc:9092")
    .option("subscribe", "building-jy")
    .option("kafka.group.id", "my_consumer_group")
    .load()
)  # 밀리초 단위 에포치 시간endingTimestamp
building_sdf = building_sdf.selectExpr("CAST(value AS STRING)", "CAST(timestamp AS STRING)")
building_sdf = building_sdf.withColumnRenamed("timestamp", "createTime")
building_sdf = building_sdf.withColumn("value", from_json(building_sdf["value"], schema3))


for field in schema3.fields:
    building_sdf = building_sdf.withColumn(field.name, building_sdf["value." + field.name])
building_sdf = building_sdf.drop("value")
# 이거쓰면 df가 repartition_num 수만큼 쪼개져서 병렬처리가능한 상태가 됨.
building_sdf = building_sdf.repartition(repartition_num)


building_sdf.printSchema()

root
 |-- createTime: string (nullable = true)
 |-- 건물번호: string (nullable = true)
 |-- 건물유형: string (nullable = true)
 |-- 연면적(m2): string (nullable = true)
 |-- 냉방면적(m2): string (nullable = true)
 |-- 태양광용량(kW): string (nullable = true)
 |-- ESS저장용량(kWh): string (nullable = true)
 |-- PCS용량(kW): string (nullable = true)



In [3]:
building_sdf = building_sdf.select(
    building_sdf["createTime"],
    building_sdf["건물번호"],
    building_sdf["건물유형"],
    building_sdf["연면적(m2)"],
    building_sdf["냉방면적(m2)"],
    building_sdf["태양광용량(kW)"],
    building_sdf["ESS저장용량(kWh)"],
    building_sdf["PCS용량(kW)"]
) \
.withColumn("createTime", building_sdf["createTime"].cast(StringType())) \
.withColumn("건물번호", building_sdf["건물번호"].cast(IntegerType())) \
.withColumn("건물유형", building_sdf["건물유형"].cast(StringType())) \
.withColumn("연면적(m2)", building_sdf["연면적(m2)"].cast(DoubleType())) \
.withColumn("냉방면적(m2)", building_sdf["냉방면적(m2)"].cast(DoubleType())) \
.withColumn("태양광용량(kW)", building_sdf["태양광용량(kW)"].cast(DoubleType())) \
.withColumn("ESS저장용량(kWh)", building_sdf["ESS저장용량(kWh)"].cast(IntegerType())) \
.withColumn("PCS용량(kW)", building_sdf["PCS용량(kW)"].cast(IntegerType()))

building_sdf.show()

+--------------------+--------+--------------+----------+------------+--------------+----------------+-----------+
|          createTime|건물번호|      건물유형|연면적(m2)|냉방면적(m2)|태양광용량(kW)|ESS저장용량(kWh)|PCS용량(kW)|
+--------------------+--------+--------------+----------+------------+--------------+----------------+-----------+
|2024-05-28 13:57:...|      72|        연구소|   31503.0|     12667.0|          95.0|            null|       null|
|2024-05-28 13:34:...|      83|  지식산업센터| 199116.93|    99603.87|          null|            null|       null|
|2024-05-28 13:57:...|      73|        연구소| 109894.22|     87915.2|         60.45|            null|       null|
|2024-05-28 13:34:...|      15|      건물기타| 167012.31|   167012.31|          null|            null|       null|
|2024-05-28 13:34:...|      33|    데이터센터|   28059.0|     20397.0|          null|            null|       null|
|2024-05-28 13:34:...|      84|  지식산업센터|  174408.2|    109973.1|          null|            null|       null|
|2024-05-28 13:34:

In [4]:

# 그냥 가져오기
train_sdf = (
    spark.read.format("kafka")
    .option("kafka.bootstrap.servers", "my-cluster-kafka-bootstrap.kafka.svc:9092")
    .option("subscribe", "test-jy")
    .option("kafka.group.id", "my_consumer_group")
    .load()
)  # 밀리초 단위 에포치 시간endingTimestamp

train_sdf = train_sdf.selectExpr("CAST(value AS STRING)", "CAST(timestamp AS STRING)")
train_sdf = train_sdf.withColumnRenamed("timestamp", "createTime")
train_sdf = train_sdf.withColumn("value", from_json(train_sdf["value"], schema))


for field in schema.fields:
    train_sdf = train_sdf.withColumn(field.name, train_sdf["value." + field.name])
train_sdf = train_sdf.drop("value")
# 이거쓰면 df가 repartition_num 수만큼 쪼개져서 병렬처리가능한 상태가 됨.


train_sdf = train_sdf.repartition(repartition_num)

train_sdf.printSchema()

train_sdf = train_sdf.select(
    train_sdf["createTime"],
    train_sdf["num_date_time"],
    train_sdf["건물번호"],
    train_sdf["일시"],
    train_sdf["기온(C)"],
    train_sdf["강수량(mm)"],
    train_sdf["풍속(m/s)"],
    train_sdf["습도(%)"],
    train_sdf["일조(hr)"],
    train_sdf["일사(MJ/m2)"],
    train_sdf["전력소비량(kWh)"],
) \
.withColumn("createTime", train_sdf["createTime"].cast(StringType())) \
.withColumn("num_date_time", train_sdf["num_date_time"].cast(StringType())) \
.withColumn("건물번호", train_sdf["건물번호"].cast(IntegerType())) \
.withColumn("일시", train_sdf["일시"].cast(StringType())) \
.withColumn("기온(C)", train_sdf["기온(C)"].cast(DoubleType())) \
.withColumn("강수량(mm)", train_sdf["강수량(mm)"].cast(DoubleType())) \
.withColumn("풍속(m/s)", train_sdf["풍속(m/s)"].cast(DoubleType())) \
.withColumn("습도(%)", train_sdf["습도(%)"].cast(IntegerType())) \
.withColumn("일조(hr)", train_sdf["일조(hr)"].cast(DoubleType())) \
.withColumn("일사(MJ/m2)", train_sdf["일사(MJ/m2)"].cast(DoubleType())) \
.withColumn("전력소비량(kWh)", train_sdf["전력소비량(kWh)"].cast(DoubleType()))

train_sdf.show()

root
 |-- createTime: string (nullable = true)
 |-- num_date_time: string (nullable = true)
 |-- 건물번호: string (nullable = true)
 |-- 일시: string (nullable = true)
 |-- 기온(C): string (nullable = true)
 |-- 강수량(mm): string (nullable = true)
 |-- 풍속(m/s): string (nullable = true)
 |-- 습도(%): string (nullable = true)
 |-- 일조(hr): string (nullable = true)
 |-- 일사(MJ/m2): string (nullable = true)
 |-- 전력소비량(kWh): string (nullable = true)



+--------------------+--------------+--------+-----------+-------+----------+---------+-------+--------+-----------+---------------+
|          createTime| num_date_time|건물번호|       일시|기온(C)|강수량(mm)|풍속(m/s)|습도(%)|일조(hr)|일사(MJ/m2)|전력소비량(kWh)|
+--------------------+--------------+--------+-----------+-------+----------+---------+-------+--------+-----------+---------------+
|2024-05-28 13:49:...|22_20220824 12|      22|20220824 12|   26.3|      null|      1.8|     68|     0.6|       2.02|        1553.76|
|2024-05-28 13:50:...|67_20220604 23|      67|20220604 23|   19.0|      null|      3.7|     75|    null|       null|        1021.92|
|2024-05-28 13:50:...|98_20220703 03|      98|20220703 03|   24.5|      null|      1.0|     78|    null|       null|         449.28|
|2024-05-28 06:45:...|30_20220616 18|      30|20220616 18|   23.6|      null|      0.6|     57|     1.0|       1.28|          944.7|
|2024-05-28 06:45:...|49_20220726 06|      49|20220726 06|   24.0|      null|      0.0|     9

In [5]:
train_sdf = train_sdf.drop("createTime")
building_sdf = building_sdf.drop("createTime")

In [6]:

def preprocess_x(df):
    to_remove_columns = ['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)']
    df = df.fillna(0)
    
    # 시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
    df = df.withColumn('month', substring('일시', 5, 2).cast(IntegerType()))
    df = df.withColumn('day', substring('일시', 7, 2).cast(IntegerType()))
    df = df.withColumn('time', substring('일시', 10, 2).cast(IntegerType()))

    df = df.join(building_sdf.select('건물번호', '건물유형', '연면적(m2)'), on='건물번호', how='left')
    df = df.dropDuplicates()
    
    # '건물유형'을 카테고리형 코드로 변환
    building_type_indexer = StringIndexer(inputCol='건물유형', outputCol='건물유형_index')
    df = building_type_indexer.fit(df).transform(df)
    df = df.drop('건물유형').withColumnRenamed('건물유형_index', '건물유형')
    
    # 불필요한 컬럼 삭제
    for c in to_remove_columns:
        if c in df.columns:
            df = df.drop(c)
            
    df.show(20, truncate=False)
    return df


In [7]:
before_split_sdf = preprocess_x(train_sdf)

+--------+-------+----------+---------+-------+---------------+-----+---+----+-----------+--------+
|건물번호|기온(C)|강수량(mm)|풍속(m/s)|습도(%)|전력소비량(kWh)|month|day|time|연면적(m2) |건물유형|
+--------+-------+----------+---------+-------+---------------+-----+---+----+-----------+--------+
|33      |24.9   |0.0       |3.4      |59     |8474.4         |6    |2  |18  |28059.0    |11.0    |
|35      |31.2   |0.0       |3.7      |70     |2258.7         |8    |1  |17  |9736.0     |11.0    |
|97      |19.4   |0.0       |4.0      |90     |954.36         |6    |6  |21  |55144.67   |10.0    |
|4       |18.6   |0.1       |2.7      |94     |734.22         |6    |16 |2   |74312.98   |0.0     |
|20      |26.1   |0.2       |1.4      |95     |1064.16        |8    |1  |23  |1.2872877E7|1.0     |
|61      |28.5   |0.0       |3.0      |78     |3724.2         |7    |7  |9   |667131.0   |6.0     |
|91      |26.5   |1.1       |1.6      |99     |1766.16        |7    |7  |20  |63865.0    |9.0     |
|75      |26.2   |0.0    

In [8]:

before_split_sdf = before_split_sdf.select(
    before_split_sdf["건물번호"],
    before_split_sdf["기온(C)"],
    before_split_sdf["강수량(mm)"],
    before_split_sdf["풍속(m/s)"],
    before_split_sdf["습도(%)"],
    before_split_sdf["전력소비량(kWh)"],
    before_split_sdf["month"],
    before_split_sdf["day"],
    before_split_sdf["time"],
    before_split_sdf["연면적(m2)"],
    before_split_sdf["건물유형"],
    
) \
.withColumn("건물번호", before_split_sdf["건물번호"].cast(StringType())) \
.withColumn("기온(C)", before_split_sdf["기온(C)"].cast(StringType())) \
.withColumn("강수량(mm)", before_split_sdf["강수량(mm)"].cast(StringType())) \
.withColumn("풍속(m/s)", before_split_sdf["풍속(m/s)"].cast(StringType())) \
.withColumn("습도(%)", before_split_sdf["습도(%)"].cast(StringType())) \
.withColumn("전력소비량(kWh)", before_split_sdf["전력소비량(kWh)"].cast(StringType())) \
.withColumn("month", before_split_sdf["month"].cast(StringType())) \
.withColumn("day", before_split_sdf["day"].cast(StringType())) \
.withColumn("time", before_split_sdf["time"].cast(StringType())) \
.withColumn("연면적(m2)", before_split_sdf["연면적(m2)"].cast(StringType())) \
.withColumn("건물유형", before_split_sdf["건물유형"].cast(StringType()))

In [13]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from datetime import datetime
import time


# 스파크 데이터프레임을 판다스 데이터프레임으로 변환
pandas_df = before_split_sdf.toPandas()

pandas_df = pandas_df.where(pd.notnull(pandas_df), None)

pandas_df = pandas_df.rename(columns={
    '건물번호': 'building_number',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '전력소비량(kWh)': 'power_consumption',
    '연면적(m2)': 'total_area',
    '건물유형': 'building_type',
})

In [14]:
pandas_df.head()

,building_number,temperature,rainfall,windspeed,humidity,power_consumption,month,day,time,total_area,building_type
0,33,24.9,0.0,3.4,59,8474.4,6,2,18,28059.0,11.0
1,35,31.2,0.0,3.7,70,2258.7,8,1,17,9736.0,11.0
2,97,19.4,0.0,4.0,90,954.36,6,6,21,55144.67,10.0
3,4,18.6,0.1,2.7,94,734.22,6,16,2,74312.98,0.0
4,20,26.1,0.2,1.4,95,1064.16,8,1,23,1.2872877E7,1.0


In [15]:
# InfluxDB 클라이언트 설정
bucket = "electric"
org = "influxdata"
token = "LQcRh6vq3wXU_QWpYFEhmk2IRZxgIn04ByEYHWW6WZ9xhNwyQB-2K6K_faA-CzvWlreXT0EES1Xz10STwAu0hQ=="
url = "http://155.230.34.52:32145/"

client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api()

In [16]:

# 판다스 데이터프레임을 InfluxDB에 적재
for index, row in pandas_df.iterrows():
    
    year = 2022
    # month, day, time 컬럼에서 값 추출
    month = int(row['month']) if row['month'] is not None else 1
    day = int(row['day']) if row['day'] is not None else 1
    hour = int(row['time']) if row['time'] is not None else 0
    
    # datetime 객체로 변환
    dt_obj = datetime(year, month, day, hour)
    
    # Unix 타임스탬프 나노초 단위로 변환
    timestamp_ns = int(time.mktime(dt_obj.timetuple()) * 1e9)
    
    # JSON 형태의 필드 값 생성
    fields_value = {
        "building_number": row['building_number'],
        "temperature": float(row['temperature']) if row['temperature'] is not None else None,
        "rainfall": float(row['rainfall']) if row['rainfall'] is not None else None,
        "windspeed": float(row['windspeed']) if row['windspeed'] is not None else None,
        "humidity": float(row['humidity']) if row['humidity'] is not None else None,
        "power_consumption": float(row['power_consumption']) if row['power_consumption'] is not None else None,
        "month": float(row['month']) if row['month'] is not None else None,
        "day": float(row['day']) if row['day'] is not None else None,
        "time": float(row['time']) if row['time'] is not None else None,
        "sunshine": float(row['total_area']) if row['total_area'] is not None else None,
        "building_type": float(row['building_type']) if row['building_type'] is not None else None
    }
    
    
    point = Point("electric_dataset") \
        .tag("building_number", row['building_number']) \
        .field("value", json.dumps(fields_value)) \
        .time(timestamp_ns, WritePrecision.NS)
    
    write_api.write(bucket=bucket, org=org, record=point)

write_api.__del__()
client.__del__()

In [ ]:
client.close()